In [1]:
import os
import pandas as pd
from matplotlib import pyplot as plt
# import tensorflow_io as tfio
import random
from pathlib import Path
import librosa
import librosa.display
import numpy as np
from statistics import mean

In [2]:
FRAME_LENGTH = 1024
HOP_LENGTH = 512
sr = 8000

In [3]:
data = pd.read_json('../4_accent_features_8k.json')

In [4]:
_ = np.array(data.mfcc)
_.shape, _

((6183,),
 array([list([[-443.1603393555, -422.7786865234, -423.5948181152, -425.0028991699, -425.8316040039, -424.9036560059, -420.4273376465, -416.5043945312, -417.8817443848, -415.5581665039, -404.6181945801, -408.2300720215, -411.9808349609, -414.1010742188, -415.9378662109, -412.93359375, -379.8030090332, -326.5506896973, -334.5718688965, -339.6754150391, -337.5093994141, -330.4769592285, -325.3367004395, -339.9404907227, -365.1604003906, -290.1697387695, -232.6450500488, -193.7247009277, -177.0232086182, -199.5745697021, -205.0437927246, -212.6782073975, -171.4652862549, -153.7566986084, -147.9802246094, -149.6708679199, -120.6398010254, -81.94871521, -62.4579658508, -92.2890777588, -123.4610595703, -84.6687774658, -81.8625488281, -112.9985733032, -108.2487945557, -160.813369751, -226.5630493164, -138.0972900391, -108.2551345825, -150.7850189209, -158.4488830566, -178.8067169189, -232.3307037354, -119.5159759521, -48.9715766907, -82.6450576782, -203.3560791016, -186.9863586426, -

In [5]:
data = data.drop(['rms', 'zrc', 'sb', 'sc'], axis=1)
data

,lable,mfcc
0,bangla,"[[-443.1603393555, -422.7786865234, -423.59481..."
1,malayalam,"[[-428.3286437988, -393.4295654297, -394.17318..."
2,malayalam,"[[-437.4763183594, -409.4303588867, -412.71850..."
3,odiya,"[[-434.8092041016, -401.2186584473, -400.85339..."
4,bangla,"[[-399.463104248, -372.8232116699, -373.531829..."
...,...,...
6178,telugu,"[[-332.5925598145, -184.8822174072, -165.35314..."
6179,odiya,"[[-411.5061035156, -379.0005493164, -384.05307..."
6180,malayalam,"[[-448.7952880859, -421.9195251465, -422.11801..."
6181,bangla,"[[-453.154876709, -426.0098266602, -424.723693..."


In [6]:
lst = []
for i in data.mfcc:
#     if 50<len(i[0])<100:
    lst.append(len(i[0]))
max(lst), min(lst), mean(lst), np.std(np.array(lst))

(426, 7, 82.67200388161086, 24.820438197834253)

In [7]:
lst = [np.array(l) for l in data.mfcc]
new_lst = []
for l in lst:
    l = np.array([np.mean(values) for values in l])
    new_lst.append(l.reshape(1, -1))
new_lst = np.array(new_lst)
new_lst.shape

(6183, 1, 40)

In [8]:
### Split the dataset into independent and dependent dataset
# mfcc_data = np.array([np.array(lst) for lst in data.mfcc])
X=new_lst
y=np.array(data['lable'].tolist())

In [9]:
X, X.shape

(array([[[-241.31078349,   74.78038768,  -27.19757481, ...,
            -2.96196981,   -1.02676847,   -0.59364521]],
 
        [[-299.20824513,  103.84525279,  -24.70479882, ...,
            -1.35972043,   -3.31951162,    0.58150785]],
 
        [[-345.31385513,   95.96350739,   -5.76264615, ...,
            -3.77577033,   -3.3540045 ,   -1.43347244]],
 
        ...,
 
        [[-297.16506285,   85.25231346,  -16.17375642, ...,
            -1.81673159,   -1.48504027,   -0.4250383 ]],
 
        [[-276.13309225,   87.86059828,  -24.54401334, ...,
             1.41364662,   -3.73048851,   -2.88456962]],
 
        [[-311.04617215,  116.43488611,   -5.674293  , ...,
             2.87672767,    2.32319292,    0.45533338]]]),
 (6183, 1, 40))

In [10]:
y

array(['bangla', 'malayalam', 'malayalam', ..., 'malayalam', 'bangla',
       'telugu'], dtype='<U9')

In [11]:
### Label Encoding
###y=np.array(pd.get_dummies(y))
### Label Encoder
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
y=to_categorical(labelencoder.fit_transform(y))

2023-03-20 23:42:05.073961: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-20 23:42:05.440648: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-20 23:42:05.440711: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-20 23:42:07.275399: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [12]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [13]:
X_train.shape

(4946, 1, 40)

In [14]:
y

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       ...,
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.]], dtype=float32)

In [15]:


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten, LSTM
from tensorflow.keras.optimizers import Adam
from sklearn import metrics



In [16]:
num_labels=y.shape[1]

In [17]:
input_shape=(1, 40)
model = Sequential()
model.add(LSTM(100,input_shape=input_shape))
model.add(Dropout(0.2))
model.add(Dense(100, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(num_labels, activation='softmax'))
model.summary()

2023-03-20 23:42:15.651781: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-20 23:42:15.652316: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-20 23:42:15.652614: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-03-20 23:42:15.652802: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-03-20 23:42:15.653038: W tensorf

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense (Dense)               (None, 100)               10100     
                                                                 
 dense_1 (Dense)             (None, 64)                6464      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dropout_2 (Dropout)         (None, 32)                0

In [18]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')

In [19]:
## Trianing my model
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime 

num_epochs = 25
num_batch_size = 32

checkpointer = ModelCheckpoint(filepath='saved_models/audio_classification.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(X_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Epoch 1/25
153/155 [============================>.] - ETA: 0s - loss: 0.8398 - accuracy: 0.6552
Epoch 1: val_loss improved from inf to 0.21720, saving model to saved_models/audio_classification.hdf5
155/155 [==============================] - 11s 23ms/step - loss: 0.8345 - accuracy: 0.6573 - val_loss: 0.2172 - val_accuracy: 0.9272
Epoch 2/25
155/155 [==============================] - ETA: 0s - loss: 0.2967 - accuracy: 0.8941
Epoch 2: val_loss improved from 0.21720 to 0.06246, saving model to saved_models/audio_classification.hdf5
155/155 [==============================] - 2s 13ms/step - loss: 0.2967 - accuracy: 0.8941 - val_loss: 0.0625 - val_accuracy: 0.9895
Epoch 3/25
151/155 [============================>.] - ETA: 0s - loss: 0.1448 - accuracy: 0.9532
Epoch 3: val_loss improved from 0.06246 to 0.05232, saving model to saved_models/audio_classification.hdf5
155/155 [==============================] - 2s 13ms/step - loss: 0.1445 - accuracy: 0.9537 - val_loss: 0.0523 - val_accuracy: 0.983

In [20]:
test_accuracy=model.evaluate(X_test,y_test,verbose=1)
print(test_accuracy[1])

39/39 [==============================] - 0s 6ms/step - loss: 0.0237 - accuracy: 0.9943
0.9943411350250244


In [21]:
X_test[1]

array([[-3.54847505e+02,  1.00387128e+02, -1.43760489e+01,
         8.87675391e+00, -1.45499848e+01, -6.56826665e+00,
        -1.14670449e+01, -6.39002056e-01, -1.33811143e+01,
         7.48432776e+00, -1.18953439e+01, -2.16107903e+00,
        -1.99738546e+01, -3.40676268e+00, -1.08832995e+01,
        -4.23590105e+00, -1.03168089e+01, -2.15638166e+00,
        -8.74021269e+00, -1.64149393e+00, -5.07332843e+00,
         1.35810460e-01, -2.19451958e+00,  6.53603858e+00,
         5.40096593e+00,  1.21255262e+01,  3.25013460e+00,
         2.99059040e+00, -6.40269169e+00, -4.60417650e+00,
        -5.69645478e+00, -4.83055116e+00, -3.39560809e+00,
        -2.45326604e+00, -1.40867026e+00, -1.47882664e+00,
        -2.35280867e+00, -2.12637430e+00, -1.41121301e+00,
        -1.94347769e+00]])

In [23]:
predictions = model.predict(X_test)
predictions

39/39 [==============================] - 0s 5ms/step


array([[9.98953581e-01, 5.89263982e-05, 9.82632278e-04, 4.70294162e-06],
       [2.12159463e-08, 9.99997675e-01, 2.16362741e-06, 6.76973286e-08],
       [9.99994457e-01, 6.68827397e-07, 4.82355472e-06, 6.63665478e-09],
       ...,
       [9.99943256e-01, 1.23973305e-05, 4.42483542e-05, 1.58725996e-07],
       [9.99998331e-01, 1.22564231e-07, 1.50588539e-06, 1.31698663e-09],
       [9.99999046e-01, 1.18195047e-07, 8.88594172e-07, 9.43426448e-10]],
      dtype=float32)